In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.extend([ '../', '../../'])
import torch
import torch.nn as nn
import torch.optim as optim
from data import input_data
import dataloader as dl
from args import Args
from model import Het_En, Classifier, EdgePredictor, Het_classify
from smote import oversample
from train import train_smote, test_smote
import cProfile

# Set device to GPU if available, else use CPU
args = Args()
args.aminer_train()
torch.cuda.empty_cache()

Current device: NVIDIA A100 80GB PCIe


In [2]:
# data = input_data(args) 
# torch.save(data, 'data.pt')
data = torch.load('../../data/aminer/am_data.pt')
print(data)

HeteroData(
  a={
    num_nodes=20171,
    y=[20171]
  },
  p={ num_nodes=13250 },
  v={ num_nodes=18 },
  p_title_embed={ x=[13250, 128] },
  p_abstract_embed={ x=[13250, 128] },
  p_net_embed={ x=[13250, 128] },
  p_a_net_embed={ x=[13250, 128] },
  p_p_net_embed={ x=[13250, 128] },
  p_v_net_embed={ x=[13250, 128] },
  a_net_embed={ x=[20171, 128] },
  a_text_embed={ x=[20171, 128] },
  v_net_embed={ x=[18, 128] },
  v_text_embed={ x=[18, 128] },
  (a, walk, a)={ edge_index=[2, 99480] },
  (a, walk, p)={ edge_index=[2, 86444] },
  (a, walk, v)={ edge_index=[2, 31716] },
  (p, walk, a)={ edge_index=[2, 65534] },
  (p, walk, p)={ edge_index=[2, 66174] },
  (p, walk, v)={ edge_index=[2, 20838] },
  (v, walk, a)={ edge_index=[2, 90] },
  (v, walk, p)={ edge_index=[2, 90] },
  (v, walk, v)={ edge_index=[2, 27] }
)


In [3]:
print(data['a','walk','p'].edge_index)
device = args.device
# data = data.to(device)

# Send all x tensors to the device
data['p_title_embed']['x'] = data['p_title_embed']['x'].to(device)
data['p_abstract_embed']['x'] = data['p_abstract_embed']['x'].to(device)
data['p_net_embed']['x'] = data['p_net_embed']['x'].to(device)
data['p_a_net_embed']['x'] = data['p_a_net_embed']['x'].to(device)
data['p_p_net_embed']['x'] = data['p_p_net_embed']['x'].to(device)
data['p_v_net_embed']['x'] = data['p_v_net_embed']['x'].to(device)
data['a_net_embed']['x'] = data['a_net_embed']['x'].to(device)
data['a_text_embed']['x'] = data['a_text_embed']['x'].to(device)
data['v_net_embed']['x'] = data['v_net_embed']['x'].to(device)
data['v_text_embed']['x'] = data['v_text_embed']['x'].to(device)

# Send all y tensors to the device
data['a']['y'] = data['a']['y'].to(device)

# Send all edge_index tensors to the device
data['a', 'walk', 'a']['edge_index'] = data['a', 'walk', 'a']['edge_index'].to(device)
data['a', 'walk', 'p']['edge_index'] = data['a', 'walk', 'p']['edge_index'].to(device)
data['a', 'walk', 'v']['edge_index'] = data['a', 'walk', 'v']['edge_index'].to(device)
data['p', 'walk', 'a']['edge_index'] = data['p', 'walk', 'a']['edge_index'].to(device)
data['p', 'walk', 'p']['edge_index'] = data['p', 'walk', 'p']['edge_index'].to(device)
data['p', 'walk', 'v']['edge_index'] = data['p', 'walk', 'v']['edge_index'].to(device)
data['v', 'walk', 'a']['edge_index'] = data['v', 'walk', 'a']['edge_index'].to(device)
data['v', 'walk', 'p']['edge_index'] = data['v', 'walk', 'p']['edge_index'].to(device)
data['v', 'walk', 'v']['edge_index'] = data['v', 'walk', 'v']['edge_index'].to(device)

edge_indices = [ data['a', 'walk', 'a'].edge_index, data['a', 'walk', 'p'].edge_index, data['a', 'walk', 'v'].edge_index ]

tensor([[    0,     0,     0,  ..., 20170, 20170, 20170],
        [10523, 10442, 10329,  ...,  4378, 12456,  6612]])


In [4]:
c_train_num = dl.train_num(data['a'].y, args.im_class_num, args.class_sample_num, args.im_ratio)
print(c_train_num, sum(c_train_num))
train_idx, val_idx, test_idx, c_num_mat = dl.segregate(data['a'].y, c_train_num)
print("train_idx: ", train_idx, len(train_idx))
print("val_idx: ", val_idx, len(val_idx))
print("test_idx: ", test_idx, len(test_idx))
# print(c_num_mat)

[50, 20, 50, 50] 170
0 335
1 202
2 299
3 289
train_idx:  [18445, 14707, 6381, 14221, 18983, 15521, 7409, 1811, 15802, 11003, 9180, 11299, 15060, 15341, 10399, 9188, 10925, 2643, 18289, 4856, 19229, 11874, 17604, 13610, 19661, 13302, 9658, 5034, 3809, 12008, 14309, 2296, 10724, 17769, 13284, 9283, 11451, 18418, 16468, 1350, 14422, 19784, 10122, 4179, 18862, 7643, 7556, 17629, 3571, 9566, 12681, 3490, 19732, 16262, 6891, 4126, 12974, 13819, 15051, 11665, 1307, 16279, 17197, 9699, 6640, 5488, 13206, 9580, 19514, 10819, 658, 2379, 16623, 9223, 6433, 10875, 9469, 13228, 4325, 12698, 4863, 7550, 9106, 14409, 205, 5582, 18747, 470, 7988, 8507, 13725, 16021, 2851, 4172, 15237, 13109, 575, 19549, 10383, 18801, 18093, 4719, 19004, 19059, 7076, 12795, 16458, 15304, 18942, 6961, 14513, 6663, 17308, 9174, 13552, 9809, 10597, 11735, 13691, 17201, 9411, 1595, 16093, 14024, 1051, 4170, 9930, 13810, 895, 3649, 17944, 17035, 3968, 3627, 15128, 10244, 20168, 14709, 9363, 18467, 17881, 12494, 12708, 13231

In [5]:
# os_mode = 'up'
# portion = sum(c_train_num)/(4*c_train_num[1])
# if os_mode != 'reweight' and os_mode != 'no':
#     if portion >= 1:
#         args.node_dim[0] = args.A_n + (portion-1)*c_train_num[1] + int((portion - int(portion))*c_train_num[1])
#     else:
#         args.node_dim[0] = args.A_n + int((portion - int(portion))*c_train_num[1])

# args.node_dim[0] += 22

In [6]:
encoder = Het_En(args.embed_dim, args.dropout)
classifier = Het_classify(args.embed_dim, args.nclass, args.dropout)
decoder_a = torch.load('pre_decoder/decoder_a.pth')
decoder_p = torch.load('pre_decoder/decoder_p.pth')
decoder_v = torch.load('pre_decoder/decoder_v.pth')
decoder_list = [decoder_a, decoder_p, decoder_v]
#print(features.shape)
encoder.to(device)
classifier.to(device)
for decoder in decoder_list:
    decoder.to(device)

## Training Part

In [7]:
torch.cuda.empty_cache()
# profiler = cProfile.Profile()
# profiler.enable()
train_smote(data, edge_indices, encoder, classifier, decoder_list, train_idx, val_idx, test_idx, args, os_mode = 'gsm', train_mode = 'noFT')
# profiler.disable()

Epoch [1/500], Loss: 1.3923, Accuracy: 0.2708, Edge Accuracy: [0.9997554988434233, 0.9996765616245255, 1.0]
Class 0: AUC-ROC- 0.5234, F1 Score- 0.0000; Class 1: AUC-ROC- 0.5005, F1 Score- 0.3176; Class 2: AUC-ROC- 0.4999, F1 Score- 0.0000; Class 3: AUC-ROC- 0.4596, F1 Score- 0.4132; Macro-Average AUC-ROC: 0.4958; Macro-Average F1 Score: 0.1827
tensor(1.3868, device='cuda:0')
Validation Loss: 1.3868, Validation Accuracy: 0.2500, Validation Edge Accuracy: []
Class 0: AUC-ROC- 0.5745, F1 Score- 0.0000; Class 1: AUC-ROC- 0.5609, F1 Score- 0.3913; Class 2: AUC-ROC- 0.6072, F1 Score- 0.0000; Class 3: AUC-ROC- 0.7683, F1 Score- 0.4000; Macro-Average AUC-ROC: 0.6277; Macro-Average F1 Score: 0.1978
tensor(1.3869, device='cuda:0')
Test Loss: 1.3869, Test Accuracy: 0.2500, Test Edge Accuracy: []
Class 0: AUC-ROC- 0.5477, F1 Score- 0.0000; Class 1: AUC-ROC- 0.5633, F1 Score- 0.4044; Class 2: AUC-ROC- 0.6448, F1 Score- 0.0000; Class 3: AUC-ROC- 0.7149, F1 Score- 0.4000; Macro-Average AUC-ROC: 0.617

In [8]:
# profiler.print_stats(sort='cumulative')  # Print results to the console
# # Or save to a file
# with open('profile_results.txt', 'w') as f:
#     profiler.dump_stats(f)

In [9]:
test_smote(data, edge_indices, encoder, classifier, decoder_list, test_idx, args = args, dataset = 'Test', os_mode = 'no', train_mode = 'noFT')

tensor(0.8430, device='cuda:0')
Test Loss: 0.8430, Test Accuracy: 0.9525, Test Edge Accuracy: []
Class 0: AUC-ROC- 1.0000, F1 Score- 0.9950; Class 1: AUC-ROC- 0.9997, F1 Score- 0.9798; Class 2: AUC-ROC- 0.9906, F1 Score- 0.9143; Class 3: AUC-ROC- 0.9982, F1 Score- 0.9519; Macro-Average AUC-ROC: 0.9971; Macro-Average F1 Score: 0.9603


(0.9525, 0.9971166666666667, 0.9602579116571655)